In [9]:
import numpy as np
from numpy.linalg import inv

In [10]:
x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 280, 280, 280, 280])

z = np.c_[x_observations, v_observations]

# Initial Conditions
a = 0  # Acceleration
v = 280 # Velocity
t = 1  # Difference in time

## CHECK HOW TO GET THE PROCESS AND OBSERVATION ERRORS.
# Process/Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6

In [11]:
z

array([[4000,  280],
       [4260,  280],
       [4550,  280],
       [4860,  280],
       [5110,  280]])

In [12]:
def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))

- `P : Estimation Covariance matrix`, with error terms that correspond to the variance of the x position and x velocity, specific to the estimates.
- For each observation that was provided, iterating through a series of processes to update the state matrix with values provided by the Kalman filter:
 - Firstly, make a prediction of where the robot will be in the next time step. This prediction is simply based on the previous position and velocity, with an acceleration parameter that adjusts the velocity. In case we dont need to adjust velocity, let it == 0
 - `A matrix` in the prediction2d function __updates the previous state based on the time that has elapsed__
 - `B matrix` __translates the acceleration into an adjustment to the position and velocity__. The formula 1/2 time squared is used to __find a distance given an acceleration__.
- Update the Process / Estimation Covariance matrix to the next time step, predicting it forward. The operation adds a time step to the matrix, subsequently updating the variance of the distance error. The A matrix is similar to the one used in predicting the State matrix values.

In [13]:
# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

In [14]:
P

array([[400,   0],
       [  0,  25]])

### Calculate Kalman gain
- Calculating the Kalman gain involved calculating the covariance matrix for the observation errors, and using it to compare with the process covariance matrix. In the problem, there’s probably more matrix rotation than what’s required, but I believe it’s meant to make the formula invariant to different matrix sizes. There is no division in matrix operations, so to find the ratio I used the dot product with the inverse of what would otherwise be the denominator.
- The __Kalman gain is a matrix of the same dimension as the inputs__, and along the __diagonal are weights__ that adjust the observed position and velocity. The lower the weights, the lower the model trusts the observations compared to the predictions.
- With the newly calculated Kalman gain:
 - Use it to weigh the difference between the observation data with the prediction, which was then used to update the state matrix. 
 - Use it to update the process covariance matrix.


In [15]:
# need to add in the angle, this problem is only for going in a straight line
for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    
    '''To simplify the problem, set off-diagonal terms to 0.'''
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    '''Calculating the Kalman Gain'''
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    '''Reshape the new data into the measurement space.'''
    Y = H.dot(data).reshape(n, -1)

    '''Update the State Matrix. Combination of the predicted state, measured values, covariance matrix and Kalman Gain'''
    X = X + K.dot(Y - H.dot(X))

    '''Update Process Covariance Matrix'''
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

# Retuen the distance and velocity
print("Kalman Filter State Matrix:\n", X)

Kalman Filter State Matrix:
 [[5116.5638995]
 [ 280.       ]]


In [ ]:
motion based on odometer

three delta: 2 angels and straight distance
     orginal angel | distance to point | angel 
        
control: imu
    
    x,y theta
    control: odometer - where we should be